<a href="https://colab.research.google.com/github/ThakkarHarsh/Amazon-Personalize/blob/main/Amazon_Personalize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import boto3
import os
import json
import numpy as np
import pandas as pd
import time
os.environ['AWS_DEFAULT_REGION'] = 'ap-northeast-1'
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

In [ ]:
!pip install awscli

In [ ]:
!aws configure

In [ ]:
bucket = "qw2" 

In [ ]:
!wget -N http://files.grouplens.org/datasets/movielens/ml-100k.zip
!unzip -o ml-100k.zip
data = pd.read_csv('./ml-100k/u.data', sep='\t', names=['USER_ID', 'ITEM_ID', 'RATING', 'TIMESTAMP'])
pd.set_option('display.max_rows', 5)
data

In [ ]:
s3 = boto3.resource('s3',
         aws_access_key_id='acces key',
         aws_secret_access_key= 'secret key')

In [ ]:
data = data[['USER_ID', 'ITEM_ID', 'TIMESTAMP']] # select columns that match the columns in the schema below
data.to_csv("extra2.csv", index=False)

s3.Bucket(bucket).Object("Intercton.csv" ).upload_file("extra2.csv")

In [ ]:
k = pd.read_csv("/content/Book1.csv")
k.to_csv("user.csv", index=False)
s3.Bucket(bucket).Object("user.csv").upload_file("user.csv")

In [ ]:
schema = {
  "type": "record",
  "name": "Users",
  "schema":"Users",
  "namespace": "com.amazonaws.personalize.schema",
 
  "fields": [
      {
          "name": "USER_ID",
          "type": "string"
      },
      {
          "name": "AGE",
          "type": "int"
      },
      {
          "name": "GENDER",
          "type": "string",
          "categorical": True
      }
  ],
  "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "x0schema",
    schema = json.dumps(schema))

schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

In [ ]:
schema={
  "type": "record",
  "name": "Items",
  "schema":"Items",
  "namespace": "com.amazonaws.personalize.schema",
  "fields": [
      {
          "name": "ITEM_ID",
          "type": "string"
      },
      {
          "name": "GENRE",
          "type": "string",
          "categorical": True
      }
  ],
  "version": "1.0"
}
create_schema_response = personalize.create_schema(
    name = "x01schema",
    schema = json.dumps(schema))

schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

In [ ]:
schema = {
    "type": "record",
    "name": "Interactions",
    "schema":"Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",  
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        }
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "x02schema",
    schema = json.dumps(schema))

schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

In [ ]:
create_dataset_group_response = personalize.create_dataset_group(
    name = "Advance-group"
)

dataset_group_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

In [ ]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

In [ ]:
dataset_type = "INTERACTIONS"
create_dataset_response = personalize.create_dataset(
    name = "demo-dataset",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn =  "arn:aws:personalize:ap-northeast-1:129962929409:schema/x02schema"
)

dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

In [ ]:
dataset_type ="Items"
create_dataset_response = personalize.create_dataset(
    name = "demo-item-dataset",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = "arn:aws:personalize:ap-northeast-1:129962929409:schema/x01schema"
)

dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

In [ ]:
dataset_type ="Users"
create_dataset_response = personalize.create_dataset(
    name = "demo-user-dataset",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = "arn:aws:personalize:ap-northeast-1:129962929409:schema/x0schema"
)

dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

In [ ]:
s3 = boto3.client("s3")

policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket),
                "arn:aws:s3:::{}/*".format(bucket)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket, Policy=json.dumps(policy))

In [ ]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "JOB-import-job",
    datasetArn = "arn:aws:personalize:ap-northeast-1:129962929409:dataset/Advance-group/USERS",
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket, "user.csv")
    },
    roleArn = "arn:aws:iam::129962929409:role/ExtraRole"
)

dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))